In [ ]:
import numpy as np
import pandas as pd
from scrapy.selector import Selector
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

In [138]:
driver = webdriver.Chrome()
url = "https://talk.washingtonpost.com/embed/stream?storyURL=https%3A%2F%2Fwww.washingtonpost.com%2Fopinions%2F2023%2F01%2F04%2Fheat-pumps-climate-carbon-emission-revolution%2F&v=6.16.2&ts=1699910100000&initialWidth=640&childId=comments&parentTitle=Opinion%20%7C%20Heat%20pumps%20could%20be%20a%20climate%20policy%20revolution%20—%20if%20we%20let%20them%20-%20The%20Washington%20Post&parentUrl=https%3A%2F%2Fwww.washingtonpost.com%2Fopinions%2F2023%2F01%2F04%2Fheat-pumps-climate-carbon-emission-revolution%2F"
driver.get(url)
time.sleep(5)
while True:
    try:
        # Identify and click the "Load More" button
        load_more_button = driver.find_element(By.ID, 'comments-loadMore')
        time.sleep(2)
        load_more_button.click()
        time.sleep(5)
    
    except Exception as e:
        # If the "Load More" button is not found, break out of the loop
        break

elements = driver.find_elements(By.CSS_SELECTOR, '[id^="comment"]')
comments_text = elements[0].text

comments=comments_text.split("share\n")

for i in range(len(comments)-1):
    comments[i] = comments[i].split("ago\n")[1]

comments = [c for c in comments if c!="READ MORE OF THIS CONVERSATION >"]
comments = [c.replace("\n", "") for c in comments]

for i in range(len(comments)-1):
    if comments[i][0:8]=="(Edited)":
        comments[i] = comments[i][8:]

print("Before eliminating short comments, we have a total of:", len(comments), "comments.")
        
comments_long = [c for c in comments if len(c)>=20]

print("After eliminating short comments, we have a total of:", len(comments_long), "comments.")

driver.quit()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Before eliminating short comments, we have a total of: 3040 comments.
After eliminating short comments, we have a total of: 2956 comments.


In [ ]:
import csv

with open('comments_long.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Comments'])
    for item in comments_long:
        writer.writerow([item])

#with open('comments.csv', 'w', newline='') as file:
#    writer = csv.writer(file)
#    writer.writerow(['Comments'])
#    for item in comments:
#        writer.writerow([item])

In [ ]:
import pandas as pd
pd.read_csv("comments_long.csv")

In [ ]:
!pip install bertopic

In [ ]:
from bertopic import BERTopic

# BERTopic

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_sm


In [ ]:
df = pd.DataFrame(pd.read_csv("comments_long.csv"))
l = list(df["Comments"])

In [ ]:
import pandas as pd
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

df = pd.DataFrame(pd.read_csv("comments_long.csv"))
l = list(df["Comments"])
nlp = spacy.load("en_core_web_sm")


def lemmatize_text(sentences):
    doc = nlp(sentences)
    return ' '.join([token.lemma_ for token in doc])

# Esempio di lista di documenti
documents = ["The pumps are powerful", "A powerful pump is running"]

# Applica la lemmatizzazione a ciascun documento
lemmatized_documents = [lemmatize_text(doc) for doc in l]


def remove_stopwords_from_sentences(sentences):
    # Process each sentence in the list using spaCy and remove stop words
    result = []
    for sentence in sentences:
        doc = nlp(sentence.lower())
        filtered_tokens = [token.text for token in doc if token.text not in STOP_WORDS]
        result.append(' '.join(filtered_tokens))

    return result

l = remove_stopwords_from_sentences(l)


## Lemmatize + remove stop words

In [99]:
import pandas as pd
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
#from nltk.stem import PorterStemmer
#import nltk

#nltk.download('punkt')

nlp = spacy.load('en_core_web_sm')
#ps = PorterStemmer()

df = pd.DataFrame(pd.read_csv("comments_long.csv"))
l = list(df["Comments"])

def preprocess_text(sentences):
    result = []
    for sentence in sentences:
        doc = nlp(sentence.lower())
        lemmatized_tokens = [token.lemma_ for token in doc if token.text not in STOP_WORDS]
        result.append(' '.join(lemmatized_tokens))
    return result

processed_documents = preprocess_text(l)


In [100]:
print(processed_documents[0])
print(l[0])

, reduce strain grid install heat pump ( require new building nationally ) , solar panel building . attempt clean global environment reverse global population explosion obvious factor talk . wonder " covert " censorship go corporate own mass medium global population issue ( 2.5 8 billion 1950 ) discuss . fact wapo columnist try scare , develop country have child , bad economically few child .
Well, to reduce strains on the grid both install heat pumps (should be required on all new buildings nationally), and solar panels on buildings. To attempt to clean up the global environment though reversing the global population explosion an obvious factor that is never talked about. Have to wonder how much "covert" censorship is going on in corporate owned mass media that the global population issue (from 2.5 to 8 billion since 1950) is never discussed. In fact you will see WaPo columnists try to scare us, because a few developed countries are having less children, that it is bad economically to

In [130]:
topic_model = BERTopic(embedding_model="all-MiniLM-L6-v2", min_topic_size = 30)
topics, _ = topic_model.fit_transform(processed_documents)

In [ ]:
#topic_model = BERTopic(embedding_model="all-MiniLM-L6-v2", min_topic_size = 30)
#topics, probs = topic_model.fit_transform(l)

In [135]:
topic_model.get_topic(topic=4)

[('geothermal', 0.12678525210781225),
 ('ground', 0.1033901600422775),
 ('heat', 0.05927173695996258),
 ('source', 0.05878098802087126),
 ('pump', 0.054208448716756764),
 ('system', 0.04526649936526318),
 ('drill', 0.04326102907823849),
 ('air', 0.03681500765093726),
 ('water', 0.031861818139461304),
 ('cost', 0.02948404857143739)]

In [127]:
def get_original_comment(modified_comment):
    return df["Comments"][processed_documents.index(modified_comment)]

get_original_comment(topic_model.get_representative_docs(8)[0])

'I really don’t want advice on how to spend my money, especially from someone who probably has far more than I have.I use my resources quite frugally and if the authorities want to save the planet they can do it with their own money.'

In [103]:
processed_documents.index(topic_model.get_representative_docs(0)[0])
#print(len(processed_documents))

2332

In [118]:
df_topics = pd.DataFrame({'topic': topics, 'document': df["Comments"]})
df_topics.loc[df_topics["topic"] == -1]


,topic,document
0,-1,"Well, to reduce strains on the grid both insta..."
1,-1,I’d like to replace my gas burning boiler but ...
2,-1,Better chance for a hydrogen powered one. The ...
3,-1,Stressing the electric grid more before shorin...
6,-1,I had never heard of heat pumps before moving ...
...,...,...
2945,-1,And wood is renewable. Currently wood and sola...
2946,-1,Won't be true in five years.I live in a net ze...
2948,-1,You should always have a fossil fuel alternati...
2950,-1,It would depend on how cold it gets and how we...


In [136]:
topic_model.visualize_topics()

In [137]:
topic_model.visualize_barchart()

# Chatbot

In [ ]:
!pip install gpt-index

In [ ]:
!pip install langchain